## Create results folder

In [20]:
# Import Packages
from zipfile import ZipFile
import pandas as pd
from datetime import datetime, date
import glob
import win32com.client as win32
import os
import shutil
from pywintypes import com_error
import numpy as np
#from suds.client import Client
import time
import base64
import openpyxl
from openpyxl import load_workbook
import pyodbc
import pytz

In [21]:
#adjust the number of days to offset the report date and the previous report date
k = 3
l = 4

#k = 3
#l = 4

In [24]:
################# Step 1. get the correct date ################################
# Create an offset of 2 Business days
offset_today = pd.tseries.offsets.BusinessDay(n=k)
offset_previous_day = pd.tseries.offsets.BusinessDay(n=l)
offset_collateral_day = pd.tseries.offsets.BusinessDay(n=k-2)

# initializing dates
today_date = date.today() - offset_today
prior_date = date.today() - offset_previous_day
collateral_date = date.today() - offset_collateral_day

# printing original date
print("Date's report date is: " + str(today_date))
print("Previous report date is: " + str(prior_date))
print("Date's collateral date is: " + str(collateral_date))

# Creating Timestamp
ts = pd.Timestamp(str(today_date))

Date's report date is: 2024-11-08 00:00:00
Previous report date is: 2024-11-07 00:00:00
Date's collateral date is: 2024-11-12 00:00:00


In [25]:
reportdate = today_date.strftime('%Y%m%d')
prior_reportdate = prior_date.strftime('%Y%m%d')

In [26]:
savePath = "C:\\O365SYNC\\IMCO\\RISK RM - Documents\\BarraOne\\Liquidity Test\\Daily Reports\\RM Pool LCR"
os.chdir(savePath)

In [27]:
# create a new folder to save reports
report_folder = reportdate + "\\"
folder_path = os.path.join(savePath, report_folder)
os.makedirs(folder_path)

In [28]:
folder_path

'C:\\O365SYNC\\IMCO\\RISK RM - Documents\\BarraOne\\Liquidity Test\\Daily Reports\\RM Pool LCR\\20241108\\'

In [29]:
file_list = ['Liquidity Report_'+reportdate+'_Pool LCR_RM.xlsm']

In [30]:
prior_file_list = ['Liquidity Report_Pool LCR_RM template - v2.xlsm']

In [31]:
# prior_report_folder = prior_reportdate + '\\'
# prior_folder_path = os.path.join(savePath, prior_report_folder)

In [32]:
for i in range(len(file_list)):

    # construct full file path
    destination = folder_path + file_list[i]
    source = savePath + "\\" + prior_file_list[i]

    shutil.copy(source, destination)
    print('copied', file_list[i])

copied Liquidity Report_20241108_Pool LCR_RM.xlsm


## Collateral

In [33]:
collateral_date = collateral_date.strftime('X%b-X%d-%Y').replace('X0','X').replace('X','')
collateral_date

'Nov-12-2024'

In [40]:
# prior_Lcr = 'Liquidity Report_' + prior_date.strftime('%Y%m%d') + '.xlsm'

try:
    xl = win32.Dispatch("Excel.Application")  # instantiate excel app
    xl.Application.visible = True
    xl.DisplayAlerts = False
    wb1 = xl.Workbooks.Open(folder_path+'Liquidity Report_'+reportdate+'_Pool LCR_RM.xlsm')

except com_error as e:
    print('Failure.', e)
else:
    print('Success.')

Success.


In [43]:
# update date
ws = wb1.Worksheets('Instructions')
ws.Activate()
ws.Range('F15').Value = today_date.strftime('%#m/%d/%Y')

In [44]:
collateral_wb = xl.Workbooks.Open(folder_path + 'IMCO Collateral Report - ' +collateral_date+'.xlsx')
collateral_ws = collateral_wb.Worksheets('Reporting')

c_ws = wb1.Worksheets('Collateral')
c_ws.Activate()

# Clear the contents 
end_row = c_ws.Range("X" + str(c_ws.Rows.Count)).End(-4162).Row
range_to_clear = c_ws.Range(f'O6:Z{end_row}')
range_to_clear.ClearContents()

collateral_ws.Activate()
last_row = collateral_ws.Range("J" + str(collateral_ws.Rows.Count)).End(-4162).Row
collateral_ws.Range(f'A6:L{last_row}').Copy()

c_ws.Activate()
c_ws.Range(f'O6:Z{end_row}').Select()
xl.Selection.PasteSpecial()

collateral_wb.Close()

## EQ Pool Data

In [38]:
import openpyxl as xl;

xl = win32.Dispatch("Excel.Application")  # instantiate excel app
xl.Application.visible = True
xl.DisplayAlerts = True

from openpyxl import load_workbook

wb1 = xl.Workbooks.Open(folder_path+'Liquidity Report_'+reportdate+'_Pool LCR_RM.xlsm')
wb2 = xl.Workbooks.Open(folder_path+'EQ Pool LCR.xlsx')

ws1 = wb2.Worksheets('View 1_IMCC0010_positionType ')
ws2 = wb2.Worksheets('View 1_IMCC0001_positionType ')
ws3 = wb2.Worksheets('View 1_IMCC0015_positionType ')

ws4 = wb1.Worksheets('IMCO Global Equity Pool')
ws5 = wb1.Worksheets('IMCO Canadian Equity Pool')
ws6 = wb1.Worksheets('IMCO EM Equity Pool')

# GPE
ws1.Activate()
ws1.Range(f'B1:AI{ws1.Range("B" + str(ws1.Rows.Count)).End(-4162).Row}').Copy()
ws4.Activate()

# Clear the contents 
end_row = ws4.Range("E" + str(ws4.Rows.Count)).End(-4162).Row
range_to_clear = ws4.Range(f'E1:AI{end_row}')
range_to_clear.Value = None

ws4.Range(f'E1:AI{end_row}').Select()
xl.Selection.PasteSpecial(Paste=12)

# CPE
ws2.Activate()
ws2.Range(f'B1:AI{ws2.Range("B" + str(ws2.Rows.Count)).End(-4162).Row}').Copy()
ws5.Activate()

# Clear the contents 
end_row = ws5.Range("E" + str(ws5.Rows.Count)).End(-4162).Row
range_to_clear = ws5.Range(f'E1:AI{end_row}')
range_to_clear.Value = None

ws5.Range(f'E1:AI{end_row}').Select()
xl.Selection.PasteSpecial(Paste=12)

# EMPE
ws3.Activate()
ws3.Range(f'B1:AI{ws3.Range("B" + str(ws3.Rows.Count)).End(-4162).Row}').Copy()
ws6.Activate()

# Clear the contents 
end_row = ws6.Range("E" + str(ws6.Rows.Count)).End(-4162).Row
range_to_clear = ws6.Range(f'E1:AI{end_row}')
range_to_clear.Value = None

ws6.Range(f'E1:AI{end_row}').Select()
xl.Selection.PasteSpecial(Paste=12)

True

In [39]:
wb1.Save()